In [69]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os

## Functions

In [70]:
# Одновимірні
def harmonic_function(X):
    X =X[0]
    return X**3 * (3 - X)**5 * np.sin(10 * np.pi * X)

def parametric_function(X, a=0, b=4, m=3, n=4, p=10, q=11):
    X = X[0]
    return (a - X)**m * (b - X)**n * np.sin(p * np.pi * X) * np.sin(q * np.pi * X)

# Двовимірні
def easom_function(X):
    x, y = X[0], X[1]
    return -np.cos(x) * np.cos(y) * np.exp(-((x - np.pi)**2 + (y - np.pi)**2))

def erkli_function(X):
    x, y = X[0], X[1]
    return (-20 * np.exp(-0.2 * np.sqrt(0.5 * (x**2 + y**2))) -
            np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np.pi * y))) +
            np.e + 20)
def cross_in_tray(X):
    x, y = X[0], X[1]
    expr = np.abs(np.sin(x) * np.sin(y) * np.exp(np.abs(100 - np.sqrt(x**2 + y**2) / np.pi)))
    return -0.0001 * (expr + 1) ** 0.1
def eggholder(X):
    x, y = X[0], X[1]
    term1 = -(y + 47) * np.sin(np.sqrt(np.abs(x / 2 + y + 47)))
    term2 = -x * np.sin(np.sqrt(np.abs(x - y - 47)))
    return term1 + term2
def holder_table(X):
    x, y = X[0], X[1]
    return -np.abs(np.sin(x) * np.cos(y) *
                   np.exp(np.abs(1 - np.sqrt(x**2 + y**2) / np.pi)))
def schaffer_1(X):
    x, y = X[0], X[1]
    numerator = np.sin(x**2 - y**2)**2 - 0.5
    denominator = (1 + 0.001 * (x**2 + y**2))**2
    return 0.5 + numerator / denominator
def schaffer_2(X):
    x, y = X[0], X[1]
    numerator = np.cos(np.sin(np.abs(x**2 - y**2)))**2 - 0.5
    denominator = (1 + 0.001 * (x**2 + y**2))**2
    return 0.5 + numerator / denominator

def beale_function(X):
    x, y = X[0], X[1]
    return ((1.5 - x + x * y)**2 +
            (2.25 - x + x * y**2)**2 +
            (2.625 - x + x * y**3)**2)

# Багатовимірна
def rastrigin_function(X, A=10):
    x = np.asarray(X)
    n = len(x)
    return A * n + np.sum(x**2 - A * np.cos(2 * np.pi * x))


In [71]:
functions = {
    "Harmonic": {
        "func": harmonic_function,
        "bounds": [(0, 3)],
        "opt_value": -32.957488,  
        "opt_position": 1.149904
    },
    "Parametric": {
        "func": parametric_function,
        "bounds": [(0, 4)],
        "opt_value": -130.095248,
        "opt_position": 1.952298
    },
    "Easom": {
        "func": easom_function,
        "bounds": [(-100, 100), (-100, 100)],
        "opt_value": -1.0,
        "opt_position": [np.pi, np.pi]
    },
    "Erkli": {
        "func": erkli_function,
        "bounds": [(-5, 5), (-5, 5)],
        "opt_value": 0.0,
        "opt_position": [0, 0]
    },
    "Cross-in-tray": {
        "func": cross_in_tray,
        "bounds": [(-10, 10), (-10, 10)],
        "opt_value": -2.062612,
        "opt_position": [1.34941, 1.34941]  # інші теж можливі
    },
    "Eggholder": {
        "func": eggholder,
        "bounds": [(-512, 512), (-512, 512)],
        "opt_value": -959.640662,
        "opt_position": [512, 404.2319]
    },
    "Holder Table": {
        "func": holder_table,
        "bounds": [(-10, 10), (-10, 10)],
        "opt_value": -19.208503,
        "opt_position": [8.05502, 9.66459]  # є 4 еквівалентні
    },
    "Schaffer 1": {
        "func": schaffer_1,
        "bounds": [(-100, 100), (-100, 100)],
        "opt_value": 0.0,
        "opt_position": [0, 0]
    },
    "Schaffer 2": {
        "func": schaffer_2,
        "bounds": [(-100, 100), (-100, 100)],
        "opt_value": 0.292579,
        "opt_position": [0, 1.25313]
    },
    "Rastrigin": {
        "func": rastrigin_function,
        "bounds": [(-5.12, 5.12)] * 3,  # або n змінних
        "opt_value": 0,
        "opt_position": [0] * 3
    },
    "Beale": {
        "func": beale_function,
        "bounds": [(-4.5, 4.5), (-4.5, 4.5)],
        "opt_value": 0.0,
        "opt_position": [3.0, 0.5]
    }
}


In [72]:
def decode_chromosome(chromosome, bounds, bits_per_dim):
    result = []
    for i, (a, b) in enumerate(bounds):
        start = i * bits_per_dim
        end = start + bits_per_dim
        gene = chromosome[start:end]
        dec = int("".join(str(int(g)) for g in gene), 2)
        x = a + dec * (b - a) / (2**bits_per_dim - 1)
        result.append(x)
    return np.array(result)

def generate_population(pop_size, bits_per_dim, num_dims):
    chromosome_length = bits_per_dim * num_dims
    return np.random.randint(0, 2, (pop_size, chromosome_length))

def evaluate_population(fitness_function, population, bounds, bits_per_dim):
    fitness = []
    for chrom in population:
        x = decode_chromosome(chrom, bounds, bits_per_dim)
        fitness.append(fitness_function(x))
    return np.array(fitness)

def selection(population, fitness, num_parents):
    fitness_min = np.min(fitness)
    adjusted_fitness = fitness - fitness_min
    probabilities = np.exp(-adjusted_fitness)
    probabilities += 1e-10
    probabilities /= np.sum(probabilities)
    selected_indices = np.random.choice(len(population), size=num_parents, replace=False, p=probabilities)
    return population[selected_indices]

def single_point_crossover(parent1, parent2):
    if np.random.rand() > 0.8:
        return parent1.copy(), parent2.copy()
    point = np.random.randint(1, len(parent1))
    child1 = np.concatenate([parent1[:point], parent2[point:]])
    child2 = np.concatenate([parent2[:point], parent1[point:]])
    return child1, child2

def uniform_crossover(parent1, parent2, prob=0.5):
    if np.random.rand() > 0.8:
        return parent1.copy(), parent2.copy()
    mask = np.random.rand(len(parent1)) < prob
    child1 = np.where(mask, parent1, parent2)
    child2 = np.where(mask, parent2, parent1)
    return child1, child2

def mutate(chromosome, mutation_rate=0.05):
    for i in range(len(chromosome)):
        if np.random.rand() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

def genetic_algorithm(fitness_function, bounds, bits_per_dim=32, pop_size=20, generations=100,
                      elite_frac=0.1, patience=20, extremum=None, tolerance=1e-6):
    num_dims = len(bounds)
    population = generate_population(pop_size, bits_per_dim, num_dims)
    elite_count = max(1, int(elite_frac * pop_size))
    best_values = []
    all_values = []

    best_score = np.inf
    no_improve_counter = 0

    for i, gen in enumerate(range(generations)):
        fitness = evaluate_population(fitness_function, population, bounds, bits_per_dim)
        sorted_indices = np.argsort(fitness)
        elites = population[sorted_indices[:elite_count]]
        best_idx = sorted_indices[0]
        best_x = decode_chromosome(population[best_idx], bounds, bits_per_dim)
        best_f = fitness[best_idx]
        best_values.append((best_x, best_f))

        all_x = np.array([decode_chromosome(ind, bounds, bits_per_dim) for ind in population])
        all_values.append(all_x)

        if extremum is not None and abs(best_f - extremum) <= tolerance:
            print(f"Target extremum {extremum} reached with tolerance {tolerance} at generation {gen}")
            break

        if best_f < best_score:
            best_score = best_f
            no_improve_counter = 0
        else:
            no_improve_counter += 1

        if no_improve_counter >= patience:
            print(f"Early stopping at generation {gen} (no improvement)")
            break

        num_parents = pop_size - elite_count
        if num_parents % 2 != 0:
            num_parents += 1

        selected = selection(population, fitness, num_parents)
        next_population = []

        for i in range(0, pop_size - elite_count, 2):
            p1, p2 = selected[i], selected[i + 1]
            c1, c2 = uniform_crossover(p1, p2)
            next_population.append(mutate(c1))
            next_population.append(mutate(c2))

        population = np.vstack(list(elites) + next_population[:pop_size - elite_count])

    return best_values, all_values



In [ ]:
def gwo(fitness_function, bounds, num_agents=50, max_iter=100,
        patience=20, extremum=None, tolerance=1e-6):
    dim = len(bounds)
    a_decay = 2 / max_iter

    positions = np.random.rand(num_agents, dim)
    for i in range(dim):
        a, b = bounds[i]
        positions[:, i] = a + positions[:, i] * (b - a)

    alpha_pos = np.zeros(dim)
    alpha_score = np.inf

    beta_pos = np.zeros(dim)
    beta_score = np.inf

    delta_pos = np.zeros(dim)
    delta_score = np.inf

    alpha_history = []
    all_history = []

    best_score = np.inf
    no_improve_counter = 0

    for iter in range(max_iter):
        for i in range(num_agents):
            fitness = fitness_function(positions[i])

            if fitness < alpha_score:
                alpha_score = fitness
                alpha_pos = positions[i].copy()
            elif fitness < beta_score:
                beta_score = fitness
                beta_pos = positions[i].copy()
            elif fitness < delta_score:
                delta_score = fitness
                delta_pos = positions[i].copy()

        all_history.append(positions.copy())
        alpha_history.append((alpha_pos.copy(), alpha_score))

        if extremum is not None and abs(alpha_score - extremum) <= tolerance:
            print(f"Target extremum {extremum} reached with tolerance {tolerance} at iteration {iter}")
            break

        if alpha_score < best_score:
            best_score = alpha_score
            no_improve_counter = 0
        else:
            no_improve_counter += 1

        if no_improve_counter >= patience:
            print(f"Early stopping at iteration {iter} (no improvement)")
            break

        a = 2 - iter * a_decay

        for i in range(num_agents):
            for j in range(dim):
                r1, r2 = np.random.rand(), np.random.rand()
                A1 = 2 * a * r1 - a
                C1 = 2 * r2
                D_alpha = abs(C1 * alpha_pos[j] - positions[i, j])
                X1 = alpha_pos[j] - A1 * D_alpha

                r1, r2 = np.random.rand(), np.random.rand()
                A2 = 2 * a * r1 - a
                C2 = 2 * r2
                D_beta = abs(C2 * beta_pos[j] - positions[i, j])
                X2 = beta_pos[j] - A2 * D_beta

                r1, r2 = np.random.rand(), np.random.rand()
                A3 = 2 * a * r1 - a
                C3 = 2 * r2
                D_delta = abs(C3 * delta_pos[j] - positions[i, j])
                X3 = delta_pos[j] - A3 * D_delta

                positions[i, j] = (X1 + X2 + X3) / 3
                positions[i, j] = np.clip(positions[i, j], bounds[j][0], bounds[j][1])

    return alpha_history, all_history


In [74]:
import tempfile
from mpl_toolkits.mplot3d import Axes3D
temp_dir = tempfile.gettempdir()

In [75]:
def plot_optimization_summary_gif(func, bounds, best_history, all_history, extremum=None, filename="optimization_summary.gif", ):
    dim = len(bounds)
    generations = len(best_history)
    frames = []
    temp_dir = tempfile.gettempdir()

    fig = plt.figure(figsize=(14, 10))
    if dim == 2:
        ax_func = fig.add_subplot(2, 3, 1, projection='3d')
    else:
        ax_func = fig.add_subplot(2, 3, 1)
    ax_info     = fig.add_subplot(2, 3, 2)
    ax_contour  = fig.add_subplot(2, 3, 3)
    ax_curve    = fig.add_subplot(2, 3, 4)
    ax_curve20  = fig.add_subplot(2, 3, 5)
    ax_distance = fig.add_subplot(2, 3, 6)

    for gen in range(generations):
        ax_func.clear()
        ax_info.clear()
        ax_contour.clear()
        ax_curve.clear()
        ax_curve20.clear()
        ax_distance.clear()
        
        ax = ax_func
        # === Subplot 1: Function Plot (1D or 2D surface) ===
        if dim == 1:
            x = np.linspace(bounds[0][0], bounds[0][1], 1000)
            y = np.array([func(np.array([val])) for val in x])
            ax.plot(x, y, label='Objective function')
            positions = all_history[gen]
            ax.scatter(positions[:, 0], [func(p) for p in positions], alpha=0.4, s=15, color='black')
            best_point = best_history[gen]
            ax.scatter(best_point[0], best_point[1], color='yellow', edgecolor='yellow', s=30, label='Best', zorder=1)
            ax.set_title("Function Plot (1D)")
            ax.set_xlabel("x")
            ax.set_ylabel("f(x)")

        elif dim == 2:
            X = np.linspace(bounds[0][0], bounds[0][1], 100)
            Y = np.linspace(bounds[1][0], bounds[1][1], 100)
            X, Y = np.meshgrid(X, Y)
            Z = np.apply_along_axis(func, 1, np.c_[X.ravel(), Y.ravel()]).reshape(X.shape)
            ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.7)
            positions = all_history[gen]
            z_vals = np.array([func(p) for p in positions])
            ax.scatter(positions[:, 0], positions[:, 1], z_vals, color='red', s=15, alpha=0.4)
            best_point = best_history[gen][0]
            best_point = best_history[gen][0]
            best_z = func(best_point)
            ax.scatter(best_point[0], best_point[1], best_z, color='yellow', edgecolor='black', s=60, label='Best', zorder=5)
            ax.set_title("Function Plot (2D Contour)")
            ax.set_xlabel("x")
            ax.set_ylabel("y")
        else:
            ax.text(0.5, 0.5, "No visualization for dim > 2", ha='center', va='center')
            ax.set_title("Unsupported Dimension")
            ax.axis('off')

        # === Subplot 2: Info Panel ===
        ax = ax_info
        info = f"Name of function: {func.__name__}\n"
        info += f"Iteration: {gen+1}/{generations}\n"
        info += f"Best value founded: {best_history[gen][1]:.7f}\n"
        info += f"Best theoretical value: {extremum}\n"
        
        if extremum is not None:
            info += f"Distance to extremum: {abs(best_history[gen][1] - extremum):.5e}"
        ax.text(0.1, 0.5, info, fontsize=12, verticalalignment='center')
        ax.axis('off')
        ax.set_title("Info Panel")

        # === Subplot 3: Contour Plot (only for 2D) ===
        ax = ax_contour
        if dim == 2:
            ax.contourf(X, Y, Z, levels=50, cmap='plasma')
            positions = all_history[gen]
            ax.scatter(positions[:, 0], positions[:, 1], color='red', s=15, alpha=0.4)
            best_point = best_history[gen][0]
            ax.scatter(best_point[0], best_point[1], color='yellow', edgecolor='black', s=30, label='Best', zorder=1)   
            ax.set_title("Contour Plot")
        else:
            ax.text(0.5, 0.5, "Not applicable", ha='center', va='center', fontsize=14)
            ax.set_title("Contour Plot")
        ax.set_xticks([])
        ax.set_yticks([])

        # === Subplot 4: Convergence === 
        ax = ax_curve
        best_fitnesses = [score for _, score in best_history[:gen+1]]
        ax.plot(range(gen+1), best_fitnesses, marker='o', markersize=2)
        ax.set_title("Best Fitness per Iteration")
        ax.set_xlabel("Iteration")
        ax.set_ylabel("Fitness")

        # === Subplot 5: Convergence (last 20 epochs) === 
        ax = ax_curve20
        best_fitnesses = [score for _, score in best_history[:gen+1]]

        start_idx = max(0, gen - 19)
        last_epochs = list(range(start_idx, gen + 1))
        last_scores = best_fitnesses[start_idx:gen + 1]

        ax.plot(last_epochs, last_scores, marker='o',markersize=2)
        ax.set_xticks(last_epochs)
        ax.set_title("Fitness Zoomed (Last 20)")
        ax.set_xlabel("Iteration")
        ax.set_ylabel("Fitness")

        # === Subplot 6: Distance from Optimum ===
        ax = ax_distance
        if extremum is not None:
            distances = [abs(score - extremum) for _, score in best_history[:gen+1]]
            ax.plot(range(gen+1), distances, marker='o', color='red', markersize=2)
            ax.set_title("Distance to Optimum")
            ax.set_xlabel("Iteration")
            ax.set_ylabel("|f_best - f_opt|")
        else:
            ax.text(0.5, 0.5, "Extremum not provided", ha='center', va='center', fontsize=14)
            ax.set_title("Distance to Optimum")

        plt.tight_layout()
        temp_file = os.path.join(temp_dir, f"frame_{gen:03d}.png")
        fig.savefig(temp_file)
        frames.append(imageio.v2.imread(temp_file))
        plt.close(fig)

    imageio.mimsave(filename, frames, fps=10)

    for gen in range(generations):
        os.remove(os.path.join(temp_dir, f"frame_{gen:03d}.png"))

    return filename

In [76]:
i = 2
for name, info in functions.items():
    # if i != 0:
    #     i -= 1
    #     continue
    print(f"Running algorithms for: {name}")
    func = info["func"]
    bounds = info["bounds"]
    opt_value = info["opt_value"]

    # === Run Genetic Algorithm ===
    ga_best, ga_all = genetic_algorithm(
        func, bounds,
        bits_per_dim=32,
        generations=100,
        pop_size=50,
        extremum=opt_value,
        tolerance=1e-6,
        patience=np.inf
    )
    print(f"Genetic Algorithm done with {len(ga_best)} epochs and result {ga_best[-1][1]}")
    # === Run Grey Wolf Optimizer ===
    gwo_best, gwo_all = gwo(
        func, bounds,
        max_iter=100,
        extremum=opt_value,
        tolerance=1e-6,
        patience=np.inf
    )
    print(f"Grey Wolf done with {len(gwo_best)} epochs and result {gwo_best[-1][1]}")
    # === Зберегти GIF для GA ===
    ga_filename = f"gif_results/ga_{name}.gif"
    plot_optimization_summary_gif(
        func=func,
        bounds=bounds,
        best_history=ga_best,
        all_history=ga_all,
        extremum=opt_value,
        filename=ga_filename
    )
    

    # === Зберегти GIF для GWO ===
    gwo_filename = f"gif_results/gwo_{name}.gif"
    plot_optimization_summary_gif(
        func=func,
        bounds=bounds,
        best_history=gwo_best,
        all_history=gwo_all,
        extremum=opt_value,
        filename=gwo_filename
    )

    print(f"GIFs saved for {name}:")
    print(f" - GA : {ga_filename}")
    print(f" - GWO: {gwo_filename}")


Running algorithms for: Harmonic
Target extremum -32.957488 reached with tolerance 1e-06 at generation 39
Genetic Algorithm done with 40 epochs and result -32.957487655312285
Grey Wolf done with 100 epochs and result -32.957447004834876
GIFs saved for Harmonic:
 - GA : gif_results/ga_Harmonic.gif
 - GWO: gif_results/gwo_Harmonic.gif
Running algorithms for: Parametric
Genetic Algorithm done with 100 epochs and result -130.0952215830277
Grey Wolf done with 100 epochs and result -130.09515535775236
GIFs saved for Parametric:
 - GA : gif_results/ga_Parametric.gif
 - GWO: gif_results/gwo_Parametric.gif
Running algorithms for: Easom
Genetic Algorithm done with 100 epochs and result -3.241365410340551e-07
Grey Wolf done with 100 epochs and result -0.9999977158180424
GIFs saved for Easom:
 - GA : gif_results/ga_Easom.gif
 - GWO: gif_results/gwo_Easom.gif
Running algorithms for: Erkli
Genetic Algorithm done with 100 epochs and result 0.0002731393250812175
Target extremum 0.0 reached with tolera

<Figure size 640x480 with 0 Axes>

In [77]:
test1 = genetic_algorithm(harmonic_function, [(0, 3)], bits_per_dim=16, generations=100, pop_size=30, tolerance=1e-4, patience=10)

Early stopping at generation 58 (no improvement)


In [78]:
test2 = genetic_algorithm(holder_table,  [(-10, 10), (-10, 10)], bits_per_dim=32, generations=20, pop_size=100, tolerance=1e-4, patience=10)